In [1]:
!nvidia-smi

Thu Jul 11 21:57:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   32C    P0              41W / 300W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [11]:
import getpass
import subprocess
import os

# Prompt for NGC API key
ngc_api_key = getpass.getpass("NGC API Key: ")

# Log in to the Docker registry
login_command = f"echo {ngc_api_key} | docker login nvcr.io -u '$oauthtoken' --password-stdin"
login_result = subprocess.run(login_command, shell=True, capture_output=True, text=True)
print(login_result.stdout)
print(login_result.stderr)

# Check if login was successful
if login_result.returncode != 0:
    raise Exception("Docker login failed")



NGC API Key:  ········


Login Succeeded

WARNING! Your password will be stored unencrypted in /home/azureuser/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store




In [19]:
# Set environment variables
os.environ["CONTAINER_NAME"] = "llama3-8b-instruct"
os.environ["IMG_NAME"] = f"nvcr.io/nim/meta/{os.environ['CONTAINER_NAME']}:1.0.0"
os.environ["NGC_API_KEY"]="<Your NGC API key>"
os.environ["LOCAL_NIM_CACHE"] = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/mayani-gpu-ci/code/.cache/nim"

# Create the cache directory
os.makedirs(os.environ["LOCAL_NIM_CACHE"], exist_ok=True)

# Define the docker run command without -it and with -d
docker_command = [
    "docker", "run", "-d", "--rm",
    f"--name={os.environ['CONTAINER_NAME']}",
    "--gpus", "all",
    "-e", f"{os.environ['NGC_API_KEY']}",
    "-v", f"{os.environ['LOCAL_NIM_CACHE']}:/opt/nim/.cache",
    "-u", str(os.getuid()),
    "-p", "8000:8000",
    os.environ["IMG_NAME"]
]

# Execute the docker run command
result = subprocess.run(docker_command, capture_output=True, text=True)


In [15]:
import subprocess
import os
import getpass

# Prompt for NGC API key if not set
if "NGC_API_KEY" not in os.environ:
    os.environ["NGC_API_KEY"] = getpass.getpass("NGC API Key: ")

# Set environment variables
container_name = "llama3-8b-instruct"
img_name = f"nvcr.io/nim/meta/{container_name}:1.0.0"
local_nim_cache = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/mayani-gpu-ci/code/.cache/nim" # this should be the path where you want to store the cache

# Create the cache directory
os.makedirs(local_nim_cache, exist_ok=True)

# Define the docker run command without -it and with -d
docker_command = [
    "docker", "run", "-d", "--rm",
    f"--name={container_name}",
    "--gpus", "all",
    "-e", f"NGC_API_KEY={os.environ['NGC_API_KEY']}",
    "-v", f"{local_nim_cache}:/opt/nim/.cache",
    "-u", str(os.getuid()),
    "-p", "8000:8000",
    img_name
]

# Execute the docker run command
result = subprocess.run(docker_command, capture_output=True, text=True)
print("stdout:", result.stdout)
print("stderr:", result.stderr)

# Check if the container started successfully
if result.returncode == 0:
    container_id = result.stdout.strip()
    print(f"Container started successfully with ID: {container_id}")

    # Optionally, check the logs of the container
    logs_command = ["docker", "logs", container_id]
    logs_result = subprocess.run(logs_command, capture_output=True, text=True)
    print("Container logs:")
    print(logs_result.stdout)
    print(logs_result.stderr)
else:
    print("Failed to start the container")

stdout: 9a7d3f7cc0fceeb3a6dc5a80dad6e859c8bef94f9963f2e251f374102601d436

stderr: 
Container started successfully with ID: 9a7d3f7cc0fceeb3a6dc5a80dad6e859c8bef94f9963f2e251f374102601d436
Container logs:

== NVIDIA Inference Microservice LLM NIM ==

NVIDIA Inference Microservice LLM NIM Version 1.0.0
Model: nim/meta/llama3-8b-instruct

Container image Copyright (c) 2016-2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

This NIM container is governed by the NVIDIA AI Product Agreement here:
https://www.nvidia.com/en-us/data-center/products/nvidia-ai-enterprise/eula/.
A copy of this license can be found under /opt/nim/LICENSE.

The use of this model is governed by the AI Foundation Models Community License
here: https://docs.nvidia.com/ai-foundation-models-community-license.pdf.

ADDITIONAL INFORMATION: Meta Llama 3 Community License, Built with Meta Llama 3. 
A copy of the Llama 3 license can be found under /opt/nim/MODEL_LICENSE.




In [24]:
!docker container ps -a

CONTAINER ID   IMAGE                 COMMAND                CREATED             STATUS             PORTS     NAMES
32fdd3fe65a4   localhost/c3:latest   "/usr/local/bin/ice"   About an hour ago   Up About an hour             c3-progenitor


In [18]:
import requests
import json

# Define the URL and headers
url = 'http://0.0.0.0:8000/v1/completions'
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# Define the payload
payload = {
    "model": "meta/llama3-8b-instruct",
    "prompt": "Once upon a time",
    "max_tokens": 64
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())

Status Code: 200
Response Body: {'id': 'cmpl-9f5372bd74444ce09a68b6335c6f9905', 'object': 'text_completion', 'created': 1720736060, 'model': 'meta/llama3-8b-instruct', 'choices': [{'index': 0, 'text': ', there was a lovely little girl name Sophie. She was eight years old and lived in a small town in the countryside. Sophie had a big heart and always tried to do the right thing, even if it meant going against the crowd.\nOne day, Sophie discovered that her cat, Mr. Whiskers, was', 'logprobs': None, 'finish_reason': 'length', 'stop_reason': None}], 'usage': {'prompt_tokens': 5, 'total_tokens': 69, 'completion_tokens': 64}}
